In [1]:
from IPython.display import Markdown as md
import datetime
import os
try:
    os.chdir(os.path.join(os.getcwd(), '../'))
    print(os.getcwd())
except:
    pass

from ia.common.jira.connection import connect
import ia.execution.algo as algo
import ia.execution.conf.components as components
import ia.common.viz.conf.dashboard as conf

report_start_time = datetime.datetime.now()
report_timestamp = f'`Report executed at {report_start_time:%Y-%m-%d %H:%M}`'
md(report_timestamp)

# Access variables

jira_url = os.environ['JIRA_URL']
jira_username = os.environ['JIRA_USERNAME']
jira_password = os.environ['JIRA_PASSWORD']

conf_url = os.environ['CONFLUENCE_URL']
conf_username = os.environ['CONFLUENCE_USERNAME']
conf_password = os.environ['CONFLUENCE_PASSWORD']

# Report variables

space = "~lswolkien"
parent_page = "Reports"
page_title = "Execution report"
projects = {
    'DANCOE': ("COE - Sprint board", "COE Sprint"),
    'DANMR':  ("Retention - Sprint board", "Retention Sprint"),
    'DANSDP': ("SDP Core - Sprint Board", "SDP Core Platform Sprint"),
    'DANTF': ("SDP Tools & Frameworks - Sprint Board", "SDPTF Sprint"),
    'DANMIG': ("SDP Migration - Stories Board", "SDP Migration Sprint"),
    'DANMRGCORE': ("MrGreen Core Board", "MrGreen Core Sprint"),
#     "DANUNCO": ("UNO Core - Sprint board", "UNO Sprint"),
    "DANSRE": ("SRE - Sprint Board", "Week")
}

last_sprints = 3

dashboard = conf.Dashboard(conf_url, conf_username, conf_password, page_title, space, parent_page)
jira_access = connect(jira_url, basic_auth=(jira_username, jira_password))

projects_progress = {}

for project_key, _ in projects.items():
    projects_progress[project_key] = algo.active_sprint_progress(jira_access, project_key)


report_head = conf.Component(
    conf.report_head, # callable which returns content and attachments for the dashboard element
    [f"DM execution report in current sprint", "Percentage of work commited to delivered"] # arguments for callable object
)

report_execution = conf.Component(
    components.execution_report,
    [projects_progress]
)

# history of execution

squad_execution_history = []
for project_key, board_sprint in projects.items():
    board_name = board_sprint[0]
    sprint_prefix = board_sprint[1]
    sprints = algo.last_sprints(jira_access, board_name, last_sprints=last_sprints, sprint_name_prefix=sprint_prefix)
    history = algo.progress_history(jira_access, project_key, sprints)
    report_history_header = conf.Component(conf.report_heading,["h2", f"{project_key} execution history"])
    report_execution_history = conf.Component(components.history_execution_report, [history])
    
    sc_labels, sc_added, sc_removed, sc_unblocked, sc_blocked = algo.sprint_churn_history(
        jira_access, project_key, history
    )
    report_sprint_churn_history = conf.Component(
        components.sprint_churn_report, [sc_labels, sc_added, sc_removed, sc_unblocked, sc_blocked]
    )
    
    carry_over_issues = conf.Component(components.carry_over_issues, [jira_access, project_key])

    squad_execution_history.append(
        report_history_header + report_execution_history + report_sprint_churn_history + carry_over_issues
    )

history_legend = "'History of execution' - issues ('User Story', 'Task', 'Bug', 'Technical Debt') which resolution date was before sprint end date."
done_on_time_legend = "'done on time' - work items that were actually done in the sprint. Only 'Done' status counts in."
done_late_legend = "'done later' - work items that were in the sprint but finished in the next iteration(s)."
carry_over_lenegs = "'Carry over' - issues that were in more than one sprint and have not yet been Done"
sprint_churn_legend = "'Scope churn' - the number of issues in the sprint which were added, removed, unblocked and blocked in each sprint. These may be signs of problems with the product roadmap and/or adjustment of priorities between organisations, the lack of adequate pre-planning between sprints (backlog refinement), an insufficient number of stakeholders resulting in missing requirments, production incidents etc..."
report_legend = conf.Component(conf.report_legend, [
    [history_legend, done_on_time_legend, done_late_legend, sprint_churn_legend, carry_over_lenegs], 
    "The execution metrics legend:"
])
# Create or Overwrite Confluance page
dashboard.publish([
        report_head, 
        report_legend,
        report_execution, 
        *squad_execution_history,
    ])

/Users/lswolkien/Devel/EngineeringMetrics


KeyboardInterrupt: 

In [ ]:
report_end_time = datetime.datetime.now()
report_timestamp = f'` Report finished at {report_end_time:%Y-%m-%d %H:%M}, time elapsed: {report_end_time - report_start_time}`'
md(report_timestamp)

In [ ]:
import glob
files = glob.glob('*.png')
for filename in files:
    try:
        os.remove(filename)
    except:
        print("Error while deleting file : ", filename)